In [23]:
class Movie:
    def __init__(self, title=None, year=None, genres=None, plot=None, rating=None, 
                 poster=None, folder_name=None):
        self.movie_info = {
            'title': title,
            'year': year,
            'genres': genres,
            'plot': plot,
            'rating': rating,
            'poster': poster,
            'folder_name': folder_name
        }

    def __str__(self):
        return f"Movie Info:\nTitle: {self.movie_info['title']}\nYear: {self.movie_info['year']}\nGenres: {self.movie_info['genres']}\nPlot: {self.movie_info['plot']}\nRating: {self.movie_info['rating']}\nPoster: {self.movie_info['poster']}"

In [38]:
import os
import re
def directory_movie_list(directory_path):
    """
    Create a list of the movies in the directory.

    :param: (str) Path for the movies directory.
    :return: (list: str) A list of the movies in the directory.
    :raises:
    """
    directory_movie_list = []
    for folder in os.listdir(directory_path):
        get_movie_title = lambda folder_name: re.search(r"(?:\[.*\][ -]*)*(.*?)(?:\()*(\d{4})", folder_name).group(1).replace('.', ' ')
        directory_movie_list.append(get_movie_title(folder))
    return directory_movie_list

In [26]:
from imdb import Cinemagoer
def update_movie_info(movie_title):
    """
    Search for movie in IMDb and return an updated movie obj if found.
    
    :param: (str) Movie title from the directory.
    :return: (Movie) Updated movie object OR None if movie not found.
    :raises:
    """
    ia = Cinemagoer() 
    movie_search_results = ia.search_movie(movie_title)
    if movie_search_results:
        movie_info = movie_search_results[0]
        ia.update(movie_info)
    else: 
        return None

    keys_dict = {
            'title': 'title',
            'year': 'year',
            'genres': 'genres',
            'plot': 'plot outline',
            'rating': 'rating',
            'poster': 'cover url',
        }
    movie = Movie()
    for key in keys_dict:
        if keys_dict[key]in movie_info.data:
            movie.movie_info[key] = movie_info.data[keys_dict[key]] 

    return movie

In [27]:
def get_movie_list(directory_movies):
    """
    Create a list of movies with updated info.
    
    :param: (list: str) A list of the movies found in the directory.
    :return: (list: Movie) A list of movies with updated info.
    :raises:
    """
    movie_list = []

    for movie_title in directory_movies:
        movie_info = update_movie_info(movie_title)
        if movie_info:
            movie_list.append(movie_info)
    return movie_list


In [28]:
import base64
import requests

def image_url_to_base64(image_url):
    """
    Retrieve movie poster as binary image and encode it as base64 then to UTF-8 string.
    
    :param: (str) url for the movie poster.
    :return: (str) movie poster as UTF-8 string.
    :raises:
    """
    response = requests.get(image_url)
    if response.status_code == 200:
        image_data = response.content
        base64_image = base64.b64encode(image_data).decode("utf-8")
        return base64_image
    else:
        return None

In [46]:
def get_movie_md(movie):
    """
    Generate content in markdown format using data from a movie object.
    
    :param: (Movie) 
    :return: (str) movie data in markdown format.
    :raises:
    """
    movie_md = ''
    newline = lambda: ' \n '

    if movie.movie_info['title']:
        movie_md += "### {} ".format(movie.movie_info['title'])

    if movie.movie_info['year']:
        movie_md += "({})".format(movie.movie_info['year']) + newline()

    if movie.movie_info['poster']:
        base64_image = image_url_to_base64(movie.movie_info['poster']) 
        movie_md += "![Image](data:image/png;base64,{})".format(base64_image) + newline()
    
    if movie.movie_info['rating']:
        movie_md += "* **rating:** {}".format(movie.movie_info['rating']) + newline()

    if movie.movie_info['genres']:
        movie_md += "* **genres:** "
        for genre in movie.movie_info['genres']:
            movie_md += "{} ".format(genre)
        movie_md += newline()

    if movie.movie_info['plot']:
        movie_md += "* **plot:** {}".format(movie.movie_info['plot']) + newline()

    movie_md += newline()

    return  movie_md 

In [51]:
def write_md(movies, directory_path):
    """
    Write movie data to My_Movies.md file.
    
    :param: (list: Movie) 
    :return: 
    :raises:
    """
    file_path = directory_path + '\My_Movies.md'  
    content = "# My Movie Collection:  \n "

    for movie in movies:
        content += get_movie_md(movie)

    with open(file_path,'w', encoding='utf-8') as file:
        file.write(content)

    print(f"Content written to {file_path}")

In [50]:
if __name__ == "__main__":
    try:
        user_input = input("Enter your movie directory path: ")
        directory_movies= directory_movie_list(user_input)
        movies = get_movie_list(directory_movies)
        write_md(movies, user_input)    
    except Exception as e:
        print(e)

Content written to C:\Users\hkhaw\Videos\MyMovies\My_Movies.md
